In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs_LO = LOStrategy()
rs_RI = RIStrategy()
rs_RandOut = RandomOuterStrategy()
rs_RandIn = RandomInnerStrategy()
strategies={"left": rs_LO, "right": rs_RI, "outer": rs_RandOut, "inner": rs_RandIn}


def test_procedure(term, steps_lim=60):
    def local_norm(_term, reduction_strategy):
        n_steps = 0
        is_term_normalized = True

        while _term.redexes:
            _term = _term._beta_conversion(reduction_strategy)
            _term = _term._update_bound_vars()
            n_steps += 1
            if n_steps > steps_lim:
                is_term_normalized = False
                break
        return n_steps, is_term_normalized

    steps, is_norm = local_norm(term._update_bound_vars(), strategies["left"])
    print(f"LO: {steps}_{'Y' if is_norm else 'N'}")
    steps, is_norm = local_norm(term._update_bound_vars(), strategies["right"])
    print(f"RI: {steps}_{'Y' if is_norm else 'N'}")
    print("RandOuter: " + ", ".join(f"{steps}_{'Y' if is_norm else 'N'}" for steps, is_norm in (local_norm(term._update_bound_vars(), strategies["outer"]) for i in range(10))))
    print("RandInner: " + ", ".join(f"{steps}_{'Y' if is_norm else 'N'}" for steps, is_norm in (local_norm(term._update_bound_vars(), strategies["inner"]) for i in range(10))))

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
        Lambda(x, Lambda(y, y_)),
        App(
            Lambda(x, App(x_, x_)),
            Lambda(x, App(x_, x_))
        )
    )


test_procedure(term_test_1())

LO: 1_Y
RI: 61_N
RandOuter: 1_Y, 1_Y, 1_Y, 1_Y, 1_Y, 1_Y, 1_Y, 1_Y, 1_Y, 1_Y
RandInner: 61_N, 61_N, 61_N, 61_N, 61_N, 61_N, 61_N, 61_N, 61_N, 61_N


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

LO: 5_Y
RI: 5_Y
RandOuter: 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y
RandInner: 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

LO: 5_Y
RI: 5_Y
RandOuter: 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y
RandInner: 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y, 5_Y


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

LO: 4_Y
RI: 4_Y
RandOuter: 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y
RandInner: 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

LO: 15_Y
RI: 15_Y
RandOuter: 15_Y, 15_Y, 15_Y, 15_Y, 15_Y, 15_Y, 15_Y, 15_Y, 15_Y, 15_Y
RandInner: 18_Y, 22_Y, 18_Y, 16_Y, 17_Y, 18_Y, 16_Y, 18_Y, 15_Y, 15_Y


# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

LO: 4_Y
RI: 4_Y
RandOuter: 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y
RandInner: 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y, 4_Y


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

LO: 13_Y
RI: 13_Y
RandOuter: 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y
RandInner: 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y, 13_Y


# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

LO: 17_Y
RI: 17_Y
RandOuter: 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y
RandInner: 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 17_Y, 18_Y, 17_Y, 17_Y


# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

LO: 39_Y
RI: 39_Y
RandOuter: 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y
RandInner: 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y, 39_Y


# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

LO: 28_Y
RI: 30_Y
RandOuter: 28_Y, 28_Y, 28_Y, 28_Y, 28_Y, 28_Y, 28_Y, 28_Y, 28_Y, 28_Y
RandInner: 30_Y, 30_Y, 30_Y, 30_Y, 30_Y, 30_Y, 30_Y, 30_Y, 30_Y, 30_Y


# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

LO: 37_Y
RI: 37_Y
RandOuter: 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y
RandInner: 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y, 37_Y


# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

LO: 59_Y
RI: 79_Y
RandOuter: 63_Y, 63_Y, 63_Y, 60_Y, 62_Y, 62_Y, 63_Y, 63_Y, 62_Y, 62_Y
RandInner: 79_Y, 79_Y, 80_Y, 79_Y, 79_Y, 79_Y, 81_Y, 79_Y, 81_Y, 81_Y


# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

LO: 55_Y
RI: 84_Y
RandOuter: 61_Y, 59_Y, 59_Y, 60_Y, 61_Y, 61_Y, 61_Y, 57_Y, 56_Y, 59_Y
RandInner: 84_Y, 86_Y, 92_Y, 86_Y, 87_Y, 84_Y, 84_Y, 84_Y, 84_Y, 90_Y
